In [11]:
import pandas as pd
from datetime import datetime, timedelta

# todos los archivos son de 3 meses


# Carga del archivo de pruebas de velocidad
#velocidad = pd.read_csv('velocidad/1724946048259.csv')
#pqrs = pd.read_excel('velocidad/PQRSD_Formato_Servicios-2024-08-29.xlsx')
#lista = pd.read_excel('velocidad/GEN_Sitios_Activos_SI-2024-08-29.xlsx')
#dias_sin_medicion = pd.read_csv('velocidad/dias_apagado.csv')

velocidad = pd.read_csv('/content/1727885348555.csv')
pqrs = pd.read_excel('/content/PQRSD_Formato_Servicios-2024-10-02.xlsx')
lista = pd.read_excel('/content/GEN_Sitios_Activos_SI-2024-10-02.xlsx')
dias_sin_medicion = pd.read_csv('/content/NetAdvisor 2024-10-02T16_19_47.846Z.csv')


# print(dias_sin_medicion.columns)
# print(velocidad.head())
# print(velocidad.shape)
# print(velocidad.info)
# print(velocidad.dtypes)
# print(velocidad.columns)

dias_sin_medicion['DISPOSITIVO'] = dias_sin_medicion['DISPOSITIVO'].astype(str)
dias_sin_medicion['DISPOSITIVO'] = dias_sin_medicion['DISPOSITIVO'].fillna('#N/D')
dias_sin_medicion['DISPOSITIVO'] = dias_sin_medicion['DISPOSITIVO'].str.split('_').str.get(0)
dias_sin_medicion['DISPOSITIVO'] = pd.to_numeric(dias_sin_medicion['DISPOSITIVO'], errors='coerce')
dias_sin_medicion.rename(columns={'DISPOSITIVO': 'ID_Beneficiario'}, inplace=True)
dias_sin_medicion = dias_sin_medicion[['ID_Beneficiario', 'DIAS SIN MEDICION']]
# print(dias_sin_medicion.head())


#N/D

# validaciones PQRS Formato Servicios
# print(pqrs.head())
# print(pqrs.columns)

# # Dataframe CD-FALLA EN EJECUCIÓN DE PRUEBA DE VELOCIDAD 5 DÍAS CALENDARIO
datos_pqrs_falla = pqrs[['ID_Beneficiario','ID', 'Fecha_Creacion', 'Fecha_Cierre_PQRS', 'Estado', 'SUBCATEGORIA']].copy()
datos_pqrs_falla['Fecha_Creacion'] = pd.to_datetime(datos_pqrs_falla['Fecha_Creacion'])
datos_pqrs_falla['Fecha_Cierre_PQRS'] = pd.to_datetime(datos_pqrs_falla['Fecha_Cierre_PQRS'], errors='coerce')
datos_pqrs_falla['Fecha_Cierre_PQRS'] = datos_pqrs_falla['Fecha_Cierre_PQRS'].dt.date
datos_pqrs_falla = datos_pqrs_falla[datos_pqrs_falla['SUBCATEGORIA'] == 'CD-FALLA EN EJECUCIÓN DE PRUEBA DE VELOCIDAD 5 DÍAS CALENDARIO'].copy()
fecha_actual = datetime.now().date()
fecha_limite = fecha_actual - timedelta(days=5)
# print(fecha_actual)
# print(fecha_limite)
# print(datos_pqrs_falla.dtypes)
datos_pqrs_falla = datos_pqrs_falla[(datos_pqrs_falla['Fecha_Cierre_PQRS'].isna()) | (datos_pqrs_falla['Fecha_Cierre_PQRS'] > fecha_limite)]
datos_pqrs_falla = datos_pqrs_falla.rename(columns={
    'ID': 'No_PQRS_Falla',
    'Fecha_Creacion': 'Fecha_Creacion_PQRS_Falla',
    'Fecha_Cierre_PQRS':'Fecha_Cierre_PQRS_Falla',
    'Estado':'Estado_Falla',
    'SUBCATEGORIA':'SUBCATEGORIA_Falla'
})
datos_pqrs_falla.sort_values(by=['ID_Beneficiario'], ascending=[True], inplace=True)
# print(datos_pqrs_falla.shape)

# #Dataframe CD-MEDICIÓN DIRECTA DE VELOCIDAD EFECTIVA DE TRANSMISIÓN DE DATOS
datos_pqrs_medicion = pqrs[['ID_Beneficiario','ID', 'Fecha_Creacion', 'Fecha_Cierre_PQRS', 'Estado', 'SUBCATEGORIA']].copy()
datos_pqrs_medicion['Fecha_Creacion'] = pd.to_datetime(datos_pqrs_medicion['Fecha_Creacion'])
datos_pqrs_medicion['Fecha_Cierre_PQRS'] = pd.to_datetime(datos_pqrs_medicion['Fecha_Cierre_PQRS'], errors='coerce')
datos_pqrs_medicion['Fecha_Cierre_PQRS'] = datos_pqrs_medicion['Fecha_Cierre_PQRS'].dt.date
datos_pqrs_medicion = datos_pqrs_medicion[datos_pqrs_medicion['SUBCATEGORIA']== 'CD-MEDICIÓN DIRECTA DE VELOCIDAD EFECTIVA DE TRANSMISIÓN DE DATOS'].copy()
fecha_actual = datetime.now().date()
fecha_limite = fecha_actual - timedelta(days=5)
datos_pqrs_medicion = datos_pqrs_medicion[(datos_pqrs_medicion['Fecha_Cierre_PQRS'].isna()) | (datos_pqrs_medicion['Fecha_Cierre_PQRS'] > fecha_limite)]
datos_pqrs_medicion = datos_pqrs_medicion.rename(columns={
    'ID': 'No_PQRS_Velocidad',
    'Fecha_Creacion': 'Fecha_Creacion_PQRS_Velocidad',
    'Fecha_Cierre_PQRS':'Fecha_Cierre_PQRS_Velocidad',
    'Estado':'Estado_Velocidad',
    'SUBCATEGORIA':'SUBCATEGORIA_Velocidad'
})
datos_pqrs_medicion.sort_values(by=['ID_Beneficiario'], ascending=[True], inplace=True)
# print(datos_pqrs_medicion.shape)


# DataFrame Ultima ejecucion
datos_velocidad = velocidad[['Identificador beneficiario', 'Fecha de ejecucion']].copy()
datos_velocidad['Fecha de ejecucion'] = pd.to_datetime(datos_velocidad['Fecha de ejecucion']).copy()
datos_velocidad['Fecha de ejecucion'] = datos_velocidad['Fecha de ejecucion'].dt.date
datos_velocidad.sort_values(by=['Identificador beneficiario', 'Fecha de ejecucion'], ascending=[True, False], inplace=True)
velocidad_limpios = datos_velocidad.drop_duplicates(subset='Identificador beneficiario', keep='first')
# print(velocidad_limpios.shape)

# # Dataframe dias sin medicion

# # Creacion archivo base
base = lista[['Id_Beneficiario', 'Meta']]
base = base[base['Meta'].isin(['1A', '1B', 'GRUPO 2', 'GRUPO 3'])]
base.sort_values(by=['Id_Beneficiario'], ascending=[True], inplace=True)
base = pd.merge(base,datos_pqrs_falla, left_on='Id_Beneficiario', right_on='ID_Beneficiario', how='left')
base = base[['Id_Beneficiario', 'Meta', 'No_PQRS_Falla', 'Fecha_Creacion_PQRS_Falla', 'Fecha_Cierre_PQRS_Falla', 'Estado_Falla', 'SUBCATEGORIA_Falla' ]]
base['No_PQRS_Falla'] = base['No_PQRS_Falla'].fillna('#N/D')
base['Fecha_Creacion_PQRS_Falla'] = base['Fecha_Creacion_PQRS_Falla'].fillna('#N/D')
base['Fecha_Cierre_PQRS_Falla'] = base['Fecha_Cierre_PQRS_Falla'].fillna('#N/D')
base['Estado_Falla'] = base['Estado_Falla'].fillna('#N/D')
base['SUBCATEGORIA_Falla'] = base['SUBCATEGORIA_Falla'].fillna('#N/D')

# # # print(base.columns)
base = pd.merge(base,datos_pqrs_medicion, left_on='Id_Beneficiario', right_on='ID_Beneficiario', how='left')
base['No_PQRS_Velocidad'] = base['No_PQRS_Velocidad'].fillna('#N/D')
base['Fecha_Creacion_PQRS_Velocidad'] = base['Fecha_Creacion_PQRS_Velocidad'].fillna('#N/D')
base['Fecha_Cierre_PQRS_Velocidad'] = base['Fecha_Cierre_PQRS_Velocidad'].fillna('#N/D')
base['Estado_Velocidad'] = base['Estado_Velocidad'].fillna('#N/D')
base['SUBCATEGORIA_Velocidad'] = base['SUBCATEGORIA_Velocidad'].fillna('#N/D')
base = base[['Id_Beneficiario', 'Meta', 'No_PQRS_Falla', 'Fecha_Creacion_PQRS_Falla', 'Fecha_Cierre_PQRS_Falla', 'Estado_Falla', 'SUBCATEGORIA_Falla', 'No_PQRS_Velocidad','Fecha_Creacion_PQRS_Velocidad','Fecha_Cierre_PQRS_Velocidad','Estado_Velocidad','SUBCATEGORIA_Velocidad']]


base = pd.merge(base, velocidad_limpios, left_on='Id_Beneficiario', right_on='Identificador beneficiario', how='left' )
base['Fecha de ejecucion'] = base['Fecha de ejecucion'].fillna('#N/D')
base.rename(columns={'Fecha de ejecucion': 'Ultima Prueba'}, inplace=True)
base = base[['Id_Beneficiario', 'Meta', 'No_PQRS_Falla', 'Fecha_Creacion_PQRS_Falla', 'Fecha_Cierre_PQRS_Falla', 'Estado_Falla', 'SUBCATEGORIA_Falla', 'No_PQRS_Velocidad','Fecha_Creacion_PQRS_Velocidad','Fecha_Cierre_PQRS_Velocidad','Estado_Velocidad','SUBCATEGORIA_Velocidad', 'Ultima Prueba']]


base = pd.merge(base, dias_sin_medicion, left_on='Id_Beneficiario', right_on='ID_Beneficiario', how='left' )
base['DIAS SIN MEDICION'] = base['DIAS SIN MEDICION'].fillna('#N/D')

base = base[['Id_Beneficiario', 'Meta', 'No_PQRS_Falla', 'Fecha_Creacion_PQRS_Falla', 'Fecha_Cierre_PQRS_Falla', 'Estado_Falla', 'SUBCATEGORIA_Falla', 'No_PQRS_Velocidad','Fecha_Creacion_PQRS_Velocidad','Fecha_Cierre_PQRS_Velocidad','Estado_Velocidad','SUBCATEGORIA_Velocidad', 'Ultima Prueba', 'DIAS SIN MEDICION']]
base['Observaciones'] = ''
base.dtypes

# print(base.head())
base.to_excel('base.xlsx', index=False)
print("Archivo generado correctamente")


Archivo generado correctamente


In [12]:
#se quitan los salientes

base1= base[~base['Id_Beneficiario'].isin(['PRUEBA SI','SALIENTE 1A','SALIENTE 1B','SALIENTE GRUPO 2','SALIENTE GRUPO 3'])].copy()


In [13]:

# Fecha actual
fecha_actual = datetime.now().date()

#fecha_actual = '2024-09-27 00:00'
#fecha_actual = datetime.strptime(fecha_actual, '%Y-%m-%d %H:%M')

# Restar 3 días
nueva_fecha = fecha_actual - timedelta(days=3)
nueva_fecha=pd.to_datetime(nueva_fecha)
base1['Ultima Prueba'] = pd.to_datetime(base1['Ultima Prueba'], errors='coerce')
print (nueva_fecha)
print (fecha_actual)

#print (base1)
df_filtrado_1 = base1[ base1['Estado_Velocidad'].isin(["#N/D"]) & base1['Estado_Falla'].isin(["#N/D"])  & (base1['Ultima Prueba'] >= nueva_fecha) & (base1['DIAS SIN MEDICION'].isin(["#N/D"]))  ]
base2= base1[~base1['Id_Beneficiario'].isin(df_filtrado_1['Id_Beneficiario'])].copy()
base2['DIAS SIN MEDICION'] = base2['DIAS SIN MEDICION'].astype('string')
df_filtrado_2 = base2[ base2['Estado_Velocidad'].isin(["#N/D"]) & base2['Estado_Falla'].isin(["#N/D"])  & (base2['Ultima Prueba'] >= nueva_fecha) & (base2['DIAS SIN MEDICION'].isin(["1.0","2.0","3.0"]))  ]
base3= base2[~base2['Id_Beneficiario'].isin(df_filtrado_2['Id_Beneficiario'])].copy()
print (base3)


2024-09-29 00:00:00
2024-10-02
      Id_Beneficiario     Meta  No_PQRS_Falla Fecha_Creacion_PQRS_Falla  \
22              14393  GRUPO 3  61FE09-A94055       2024-06-01 07:02:51   
36              15465  GRUPO 3  5D4FF9-FE085F       2024-09-25 07:04:21   
57              22646  GRUPO 2  52993D-4017DB       2024-10-01 07:03:05   
60              22649       1A           #N/D                      #N/D   
65              22661  GRUPO 2  13CE99-4A4EA1       2024-09-23 07:03:59   
...               ...      ...            ...                       ...   
6374            74676  GRUPO 3  2DBC9A-5E53A9       2024-09-24 07:04:31   
6380            74684  GRUPO 3           #N/D                      #N/D   
6384            74689  GRUPO 3  66B4E3-ADF389       2024-09-09 07:03:35   
6401            74716  GRUPO 3  608F2B-F398EF       2024-09-21 07:03:09   
6413            75120  GRUPO 3  5CD240-CBA166       2024-09-18 07:03:00   

     Fecha_Cierre_PQRS_Falla            Estado_Falla  \
22          

In [23]:
#se inicia con dia 0 en contador

#Ultima Prueba

df_filtrado_3 = base3[(base3['Estado_Falla'].isin (["Cerrar"]))  & (base3['Estado_Velocidad'].isin (["Cerrar"]))  &  (base3['DIAS SIN MEDICION'].isin(["#N/D","1.0","2.0","3.0","4.0"])) ]
#print (df_filtrado_3)
#df_filtrado_3.describe()
base4= base3[~base3['Id_Beneficiario'].isin(df_filtrado_3['Id_Beneficiario'])].copy()
print (base4)




      Id_Beneficiario     Meta  No_PQRS_Falla Fecha_Creacion_PQRS_Falla  \
22              14393  GRUPO 3  61FE09-A94055       2024-06-01 07:02:51   
57              22646  GRUPO 2  52993D-4017DB       2024-10-01 07:03:05   
60              22649       1A           #N/D                      #N/D   
72              22674  GRUPO 3  3967D5-3EE8AF       2024-10-02 07:03:25   
77              22680  GRUPO 3  547D85-97F8D5       2024-09-26 07:03:47   
...               ...      ...            ...                       ...   
6374            74676  GRUPO 3  2DBC9A-5E53A9       2024-09-24 07:04:31   
6380            74684  GRUPO 3           #N/D                      #N/D   
6384            74689  GRUPO 3  66B4E3-ADF389       2024-09-09 07:03:35   
6401            74716  GRUPO 3  608F2B-F398EF       2024-09-21 07:03:09   
6413            75120  GRUPO 3  5CD240-CBA166       2024-09-18 07:03:00   

     Fecha_Cierre_PQRS_Falla            Estado_Falla  \
22                      #N/D  Asignado Mesa

In [17]:
#se inicia con dia 0 en contador

#Ultima Prueba

#df_filtrado_3 = base3[(base3['Estado_Falla'].isin (["#N/D"])) & (base3['Estado_Velocidad'].isin (["#N/D"])) & (base3['DIAS SIN MEDICION'] .isin (["#N/D","1.0","2.0","3.0","4.0","5.0","6.0","7.0","8.0","9.0"]) )]
#print (df_filtrado_3)
#base4= base3[~base3['Id_Beneficiario'].isin(df_filtrado_3['Id_Beneficiario'])].copy()
#print (base4)





      Id_Beneficiario     Meta No_PQRS_Falla Fecha_Creacion_PQRS_Falla  \
60              22649       1A          #N/D                      #N/D   
164             22897  GRUPO 2          #N/D                      #N/D   
189             22936  GRUPO 2          #N/D                      #N/D   
240             23041       1A          #N/D                      #N/D   
392             23333  GRUPO 3          #N/D                      #N/D   
...               ...      ...           ...                       ...   
6147            70904  GRUPO 2          #N/D                      #N/D   
6283            74287       1B          #N/D                      #N/D   
6346            74498  GRUPO 3          #N/D                      #N/D   
6355            74643  GRUPO 3          #N/D                      #N/D   
6380            74684  GRUPO 3          #N/D                      #N/D   

     Fecha_Cierre_PQRS_Falla Estado_Falla SUBCATEGORIA_Falla  \
60                      #N/D         #N/D      

In [ ]:
#validacion 3 (cerrado-cerrado o #ND<6dias)
#df_filtrado_3 = base[(base['Estado_Falla'] in ("#N/D","Cerrar")) & (base['Estado_Velocidad'] in ("#N/D","Cerrar")) & (base['DIAS SIN MEDICION'] > 6 ) ]
#total_falla_3=df_filtrado_3.shape[0]
#print ("Cantidad de PQRSD agendados con fecha maxima:")
#print (df_filtrado_3)